## Imports

In [1]:
import pandas as pd
import plotly.graph_objects as go

## Função para formatar valores no gráfico (K, M)

In [ ]:
def formata_valores(number):
    """
    Formata valores. K-mil, M-milhoes, B-bilhoes.
    Fonte: https://gist.github.com/Abdelkrim/02e604fc38e7f35969e5552f13e4af0a
    """
    for unit in ['','K','M']:
        if abs(number) < 1000.0:
            return f"{number:6.0f}{unit}".strip()
        number /= 1000.0
    return f"{number:6.1f}B".strip().replace('.0', '')

# Ridgeline plot

In [ ]:
for idx, produto in enumerate(df['nome_unificado_produto'].unique()):

    def filtro_segmento(df_, base):
        df = df_[df_['nome_unificado_produto'] == produto][['segmento_item', 'vlr_item']].assign(base = base)
        return df
    
    tmp_seg = pd.concat([filtro_segmento(alta_top_20_rio_unidade, 'rio_unidade'),
                         filtro_segmento(alta_top_20_sp_unidade, 'sp_unidade'),
                         filtro_segmento(alta_top_20_rio_movel, 'rio_movel'),
                         filtro_segmento(alta_top_20_sp_movel, 'sp_movel')])

    fig = go.Figure()

    cores = {'rio_unidade':'#1696d2',
             'sp_unidade':'#ec008b',
             'rio_movel':'#fdbf11',
             'sp_movel':'#5c5859'}

    for idx, base in enumerate(tmp_seg['base'].unique()):
        tmp_base = tmp_seg[tmp_seg.base == base].copy()
        tmp_base['cor'] = tmp_base['base'].map(cores)
        tmp_base = tmp_base.reset_index()

        _ = fig.add_trace(
                go.Violin(
                    x = tmp_base['vlr_item'],
                    name = tmp_base.base.iloc[0],
                    marker_color = tmp_base['cor'][0]
                ),
            )

    _ = fig.update_traces(orientation = 'h', 
                          side = 'positive',
                          width = 4,
                          points = False)

    _ = fig.update_layout(height = max((idx+1) * 80, 300), 
                          title = f"Distribuição dos preços por recorte do segmento [{tmp_base['segmento_item'][0]}] {produto}", 
                          title_x = 0.5,
                          xaxis_showgrid = False, 
                          xaxis_zeroline = False)
    
    fig.show()

# Distribuição - absoluto em barras e percentual em linhas

In [ ]:
# distribuição da hora de atendimento

padding = 1

# base de dados ------------------------------------------------------------------------------------

tmp = df[['horario_atendimento', 'hash_atendimento']].copy()
tmp['horario_atendimento'] = df['horario_atendimento'].apply(lambda x: x[:2])
tmp['horario_atendimento'] = tmp['horario_atendimento'].str.replace(':', "").astype(int)
tmp = tmp.groupby(by='horario_atendimento').agg({'hash_atendimento': 'nunique'}).sort_values(by='horario_atendimento')
tmp['pct'] = (tmp['hash_atendimento']/tmp['hash_atendimento'].sum()) * 100

fig, ax = plt.subplots(2,
                       figsize=(20, 6), 
                       height_ratios=[1, 2], 
                       sharex=True)

# percentual ---------------------------------------------------------------------------------------

sns.lineplot(x=tmp.index.astype(str), y=(tmp['pct']), marker='o', ax=ax[0], markersize=5,
            color = 'steelblue')
for idx, pct in (tmp['pct']).items():
    ax[0].text(str(idx), pct+padding, f'{pct:.1f}%', ha='center', size = 8)
ax[0].set_ylabel(f"pct_{'atendimentos'}")

# número absoluto ----------------------------------------------------------------------------------

sns.barplot(x=tmp.index.astype(str), 
            y=tmp['hash_atendimento'], 
            ax=ax[1],
            color = 'steelblue')
ax[1].bar_label(ax[1].containers[0], fmt = formata_valores, size = 8)
ax[1].set_ylabel(f"num_{'atendimentos'}")

plt.xlabel('Horário de atendimento')
plt.xticks(fontsize = 7)
plt.yticks(fontsize = 7)
sns.despine(bottom=True)
plt.show()

# Gráfico de pareto

In [ ]:
def pareto(column, agg_col='hash_atendimento', agg_op='nunique', ylabel='atendimentos', n=10, padding=3, size=(20,6)):
    tmp = df.groupby(by=column).agg({agg_col: agg_op}).sort_values(by=agg_col,ascending=False)
    tmp['pct'] = (tmp[agg_col]/tmp[agg_col].sum()).cumsum() * 100

    fig, ax = plt.subplots(2, figsize=size, height_ratios=[1, 2], sharex=True)

    sns.lineplot(x=tmp.head(n).index.astype(str), y=(tmp['pct']).head(n), marker='o', ax=ax[0], markersize=8, color = 'steelblue')
    for idx, pct in (tmp['pct'].head(n)).items():
        ax[0].text(str(idx), pct+padding, f'{pct:.2f}%', ha='center', size = 8)
    ax[0].set_ylabel(f'pct_{ylabel}')

    sns.barplot(x=tmp.head(n).index.astype(str), y=tmp[agg_col].head(n), ax=ax[1],
            color = 'steelblue')
    ax[1].bar_label(ax[1].containers[0], fmt=formata_valores, size = 8)
    ax[1].set_ylabel(f'num_{ylabel}')

    plt.xlabel(column)
    plt.xticks(rotation = 90, fontsize = 9)
    plt.yticks(fontsize = 7)
    sns.despine(bottom=True)
    plt.show()

# Função para encontrar pares de palavras similares (jaro-winkler)

In [ ]:
def find_similar_pairs(texts: pd.Series, threshold=0.75):
    texts = texts.astype(str)
    count_texts = texts.value_counts(dropna=False).to_dict()
    unique_texts = texts.unique()
    similar_pairs = []

    for i in range(len(unique_texts)):
        for j in range(i+1, len(unique_texts)):
            text_1 = unique_texts[i]
            text_2 = unique_texts[j]
            similarity = td.jaro(text_1, text_2)

            if similarity > threshold:
                similar_pairs.append((text_1, text_2, similarity))
            
    df_similar_pairs = pd.DataFrame(similar_pairs, columns=['text', 'similar_text', 'similarity'])
    if df_similar_pairs.empty:
        return df_similar_pairs
    
    df_similar_pairs[['text', 'similar_text']] = df_similar_pairs\
        .apply(lambda row: tuple(sorted([row['text'], row['similar_text']], key=lambda x: count_texts[x], reverse=True)), axis=1).apply(pd.Series)
    return df_similar_pairs.sort_values(['similarity'], ascending=False).reset_index(drop=True)

In [ ]:
# uso

tmp = df[['nome_unidade']].drop_duplicates()

tmp = find_similar_pairs(df['nome_unidade'], 0.95)
with pd.option_context('display.max_rows', None):
    display(tmp)

# Função com número e percentual de categorias

In [ ]:
def cont_perc_categorias(df_, coluna = 'atend_b2b2c'):
    df_temp = df_.copy()
    df_temp = df_temp[coluna].value_counts(dropna = False).reset_index()
    df_temp['perc'] = round(df_temp['count'] * 100 / df_temp['count'].sum(), 3)

    return df_temp

# Crosstab

In [ ]:
# marca

marca_faixa_renda = df.copy().sort_values('faixa_renda')[['marca', 'chave_pac', 'faixa_renda']].drop_duplicates()
marca_faixa_renda = pd.crosstab(marca_faixa_renda['marca'], marca_faixa_renda['faixa_renda'], margins = False, normalize = 'index')

plt.figure(figsize = (9, 16))
ax = sns.heatmap(marca_faixa_renda, cmap = 'Greens', annot = True, annot_kws={'size': 8}, fmt=".1%")

plt.xticks(fontsize = 8)
plt.yticks(fontsize = 8)

ax = ax